# **3. Initial Characteristic Analysis**

## **3.1 Characteristic Binning**

In [38]:
# Import library
import pandas as pd
import numpy as np

# Load configuration
import src.utils as utils

- We concat the predictors (X) & response (y) data for train set first
- Update the config file to have those concated data path.

In [54]:
config_data = utils.config_load()
config_data

{'raw_dataset_path': 'data/raw/Training Data.csv',
 'dataset_path': 'data/output/data.pkl',
 'predictors_set_path': 'data/output/predictors.pkl',
 'response_set_path': 'data/output/response.pkl',
 'train_path': ['data/output/X_train.pkl', 'data/output/y_train.pkl'],
 'test_path': ['data/output/X_test.pkl', 'data/output/y_test.pkl'],
 'data_train_path': 'data/output/data_train.pkl',
 'data_train_binned_path': 'data/output/data_train_binned.pkl',
 'crosstab_list_path': 'data/output/crosstab_list.pkl',
 'WOE_table_path': 'data/output/WOE_table.pkl',
 'IV_table_path': 'data/output/IV_table.pkl',
 'response_variable': 'risk_flag',
 'test_size': 0.3,
 'num_columns': ['income',
  'age',
  'experience',
  'current_job_years',
  'current_house_years'],
 'cat_columns': ['married',
  'house_ownership',
  'car_ownership',
  'profession',
  'city',
  'state'],
 'num_of_bins': 4}

In [55]:
def concat_data(type):
    """Concat the input (X) & output (y) data"""
    X = utils.pickle_load(config_data[f'{type}_path'][0])
    y = utils.pickle_load(config_data[f'{type}_path'][1])

    # Concatenate X & y
    data = pd.concat((X, y),
                    axis = 1)

    #Validate data
    print(f'Data shape:', data.shape)

    #Dump concatenated data
    utils.pickle_dump(data, config_data[f'data_{type}_path'])

    return data

In [56]:
#Check the function for train data
data_train = concat_data(type = 'train')
data_train.head()

Data shape: (176400, 12)


,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
203209,6840395,40,19,single,rented,yes,Librarian,Dindigul,Tamil_Nadu,9,12,0
42903,9618732,38,11,single,rented,yes,Aviator,Guwahati,Assam,11,12,0
59095,6861713,42,5,single,rented,no,Comedian,Chittoor[28],Andhra_Pradesh,5,12,0
69516,8454331,61,3,married,rented,no,Technical_writer,Rampur,Uttar_Pradesh,3,11,0
232569,2937858,76,8,married,rented,no,Civil_servant,Dhule,Maharashtra,8,14,0


1. Then we bin the concatenated data.
2. Update the config file to have:
    - The numerical column names
    - The categorical column names
    - The missing column names
    - The number of bins
    - The path for binned train set

In [57]:
config_data = utils.config_load()
config_data

{'raw_dataset_path': 'data/raw/Training Data.csv',
 'dataset_path': 'data/output/data.pkl',
 'predictors_set_path': 'data/output/predictors.pkl',
 'response_set_path': 'data/output/response.pkl',
 'train_path': ['data/output/X_train.pkl', 'data/output/y_train.pkl'],
 'test_path': ['data/output/X_test.pkl', 'data/output/y_test.pkl'],
 'data_train_path': 'data/output/data_train.pkl',
 'data_train_binned_path': 'data/output/data_train_binned.pkl',
 'crosstab_list_path': 'data/output/crosstab_list.pkl',
 'WOE_table_path': 'data/output/WOE_table.pkl',
 'IV_table_path': 'data/output/IV_table.pkl',
 'response_variable': 'risk_flag',
 'test_size': 0.3,
 'num_columns': ['income',
  'age',
  'experience',
  'current_job_years',
  'current_house_years'],
 'cat_columns': ['married',
  'house_ownership',
  'car_ownership',
  'profession',
  'city',
  'state'],
 'num_of_bins': 4}

In [58]:
# Create a function for binning the numerical predictor
def create_num_binning(data, predictor_label, num_of_bins):
    """Bin the numerical predictor"""
    # Create a new column containing the binned predictor
    data[predictor_label + "_bin"] = pd.qcut(data[predictor_label],
                                             q = num_of_bins)

    return data

In [59]:
def bin_data(type):
    """Bin the numerical data"""
    # Load the concatenated data
    data = utils.pickle_load(config_data[f'data_{type}_path'])

    # Bin the numerical columns
    num_columns = config_data['num_columns']
    num_of_bins = config_data['num_of_bins']

    for column in num_columns:
        data_binned = create_num_binning(data = data,
                                         predictor_label = column,
                                         num_of_bins = num_of_bins)

    # Validate
    print(f"Original data shape : ", data.shape)
    print(f"Binned data shape  : ", data_binned.shape)

    # Dump binned data
    utils.pickle_dump(data_binned, config_data[f'data_{type}_binned_path'])
        
    return data_binned

In [60]:
# Check the function
binned_train = bin_data(type='train')
binned_train.head()

Original data shape :  (176400, 17)
Binned data shape  :  (176400, 17)


,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag,income_bin,age_bin,experience_bin,current_job_years_bin,current_house_years_bin
203209,6840395,40,19,single,rented,yes,Librarian,Dindigul,Tamil_Nadu,9,12,0,"(5001124.0, 7475292.0]","(35.0, 50.0]","(15.0, 20.0]","(6.0, 9.0]","(11.0, 12.0]"
42903,9618732,38,11,single,rented,yes,Aviator,Guwahati,Assam,11,12,0,"(7475292.0, 9999938.0]","(35.0, 50.0]","(10.0, 15.0]","(9.0, 14.0]","(11.0, 12.0]"
59095,6861713,42,5,single,rented,no,Comedian,Chittoor[28],Andhra_Pradesh,5,12,0,"(5001124.0, 7475292.0]","(35.0, 50.0]","(-0.001, 5.0]","(3.0, 6.0]","(11.0, 12.0]"
69516,8454331,61,3,married,rented,no,Technical_writer,Rampur,Uttar_Pradesh,3,11,0,"(7475292.0, 9999938.0]","(50.0, 65.0]","(-0.001, 5.0]","(-0.001, 3.0]","(9.999, 11.0]"
232569,2937858,76,8,married,rented,no,Civil_servant,Dhule,Maharashtra,8,14,0,"(2511105.0, 5001124.0]","(65.0, 79.0]","(5.0, 10.0]","(6.0, 9.0]","(13.0, 14.0]"


### **3.2 WoE and IV**

To assess the strenght of each characteristic individually as a predictor of the credit performance.
Udapte the config file to have:
    - crosstab list path
    - WOE table path
    - IV table path

In [61]:
config_data = utils.config_load()
config_data

{'raw_dataset_path': 'data/raw/Training Data.csv',
 'dataset_path': 'data/output/data.pkl',
 'predictors_set_path': 'data/output/predictors.pkl',
 'response_set_path': 'data/output/response.pkl',
 'train_path': ['data/output/X_train.pkl', 'data/output/y_train.pkl'],
 'test_path': ['data/output/X_test.pkl', 'data/output/y_test.pkl'],
 'data_train_path': 'data/output/data_train.pkl',
 'data_train_binned_path': 'data/output/data_train_binned.pkl',
 'crosstab_list_path': 'data/output/crosstab_list.pkl',
 'WOE_table_path': 'data/output/WOE_table.pkl',
 'IV_table_path': 'data/output/IV_table.pkl',
 'response_variable': 'risk_flag',
 'test_size': 0.3,
 'num_columns': ['income',
  'age',
  'experience',
  'current_job_years',
  'current_house_years'],
 'cat_columns': ['married',
  'house_ownership',
  'car_ownership',
  'profession',
  'city',
  'state'],
 'num_of_bins': 4}

In [62]:
def create_crosstab_list():
    """Generate the crosstab list (contingency table) for WOE and IV calculation. Only in training data"""
    # load the binned train data
    data_train_binned = utils.pickle_load(config_data['data_train_binned_path'])

    # load the response variable (we will summarize based on the response variable)
    response_variable = config_data['response_variable']

    # iterate over numercial columns
    crosstab_num = []
    num_columns = config_data['num_columns']
    for column in num_columns:
        # Create a contingency table
        crosstab = pd.crosstab(data_train_binned[column + "_bin"],
                               data_train_binned[response_variable],
                               margins = True)

        # Append to the list
        crosstab_num.append(crosstab)

    # iterate over categorical columns
    crosstab_cat = []
    cat_columns = config_data['cat_columns']
    for column in cat_columns:
        # Create a contingency table
        crosstab = pd.crosstab(data_train_binned[column],
                               data_train_binned[response_variable],
                               margins = True)

        # Append to the list
        crosstab_cat.append(crosstab)

    # Put all two in a crosstab_list
    crosstab_list = crosstab_num + crosstab_cat

    # Validate the crosstab_list
    print('number of num bin : ', [bin.shape for bin in crosstab_num])
    print('number of cat bin : ', [bin.shape for bin in crosstab_cat])

    # Dump the result
    utils.pickle_dump(crosstab_list, config_data['crosstab_list_path'])

    return crosstab_list

In [63]:
# Check the function
crosstab_list = create_crosstab_list()
crosstab_list[0]

number of num bin :  [(5, 3), (5, 3), (5, 3), (5, 3), (5, 3)]
number of cat bin :  [(3, 3), (4, 3), (3, 3), (52, 3), (318, 3), (30, 3)]


risk_flag,0,1,All
income_bin,,,
"(10309.999, 2511105.0]",38469,5647,44116
"(2511105.0, 5001124.0]",38872,5224,44096
"(5001124.0, 7475292.0]",38867,5223,44090
"(7475292.0, 9999938.0]",38495,5603,44098
All,154703,21697,176400


In [64]:
crosstab_list[10]

risk_flag,0,1,All
state,,,
Andhra_Pradesh,15667,2053,17720
Assam,4334,643,4977
Bihar,11912,1804,13716
Chandigarh,436,43,479
Chhattisgarh,2345,370,2715
Delhi,3401,408,3809
Gujarat,7094,909,8003
Haryana,4852,671,5523
Himachal_Pradesh,514,72,586


In [65]:
def WOE_and_IV():
    """Get the WoE and IV"""
    # Load the crosstab list
    crosstab_list = utils.pickle_load(config_data['crosstab_list_path'])

    # Create intial storage for WoE and IV
    WOE_list, IV_list = [], []

    # Perform the calculation for all crosstab list
    for crosstab in crosstab_list:
        # Calculate the WoE and IV
        crosstab['p_good'] = crosstab[0]/crosstab[0]['All']
        crosstab['p_bad'] = crosstab[1]/crosstab[1]['All']
        crosstab['WOE'] = np.log(crosstab['p_good']/crosstab['p_bad'])
        crosstab['contribution'] = (crosstab['p_good']-crosstab['p_bad'])*crosstab['WOE']
        IV = crosstab['contribution'][:-1].sum()

        # Append to list
        WOE_list.append(crosstab)

        add_IV = {'Characteristic': crosstab.index.name,
                'Information Value': IV}
        IV_list.append(add_IV)

    # Create WOE Table
    # Create initial table to summarize the WOE values
    WOE_table = pd.DataFrame({'Characteristic': [],
                                'Attribute': [],
                                'WOE': []})
    for i in range(len(crosstab_list)):
        #Define crosstab and reset index
        crosstab = crosstab_list[i].reset_index()

        # Save the characteristic name
        char_name = crosstab.columns[0]

        # Only use two columns (Attribute name and its WOE value)
        # Drop the last row (average/total WOE)
        crosstab = crosstab.iloc[:-1, [0,-2]]
        crosstab.columns = ['Attribute', 'WOE']

        # Add the characteristic name in a column
        crosstab['Characteristic'] = char_name

        WOE_table = pd.concat((WOE_table, crosstab),
                                axis = 0)

        # Recorder the column
        WOE_table.columns = ['Characteristic',
                            'Attribute',
                            'WOE']

    # Create IV Table
    # Create the initial table for IV
    IV_table = pd.DataFrame({'Characteristic': [],
                            'Information Value' :[]})
    IV_table = pd.DataFrame(IV_list)

    # Define the predictive power of each characteristic
    strength = []

    # Assign the rule of thumb regarding IV
    for iv in IV_table['Information Value']:
        if iv < 0.02:
            strength.append('Unpredictive')
        elif iv >= 0.02 and iv < 0.1:
            strength.append('Weak')
        elif iv >= 0.1 and iv < 0.3:
            strength.append('Medium')
        else:
            strength.append('Strong')

    # Assign the strenght to each characteristic
    IV_table = IV_table.assign(Strength = strength)

    # Sort the table by the IV values
    IV_table = IV_table.sort_values(by='Information Value')

    # Validate
    print('WOE table shape : ', WOE_table.shape)
    print('IV table shape : ', IV_table.shape)

    # Dump data
    utils.pickle_dump(WOE_table, config_data['WOE_table_path'])
    utils.pickle_dump(IV_table, config_data['IV_table_path'])

    return WOE_table, IV_table

In [66]:
# Check the function
WOE_table, IV_table = WOE_and_IV()

WOE table shape :  (424, 3)
IV table shape :  (11, 3)


In [67]:
WOE_table

,Characteristic,Attribute,WOE
0,income_bin,"(10309.999, 2511105.0]",-0.045605
1,income_bin,"(2511105.0, 5001124.0]",0.042678
2,income_bin,"(5001124.0, 7475292.0]",0.042740
3,income_bin,"(7475292.0, 9999938.0]",-0.037107
0,age_bin,"(20.999, 35.0]",-0.117910
...,...,...,...
24,state,Tripura,-0.409416
25,state,Uttar_Pradesh,0.044885
26,state,Uttar_Pradesh[5],-0.090336
27,state,Uttarakhand,0.607073


In [68]:
IV_table

,Characteristic,Information Value,Strength
4,current_house_years_bin,0.000297,Unpredictive
0,income_bin,0.001776,Unpredictive
5,married,0.004158,Unpredictive
7,car_ownership,0.005417,Unpredictive
1,age_bin,0.005446,Unpredictive
3,current_job_years_bin,0.007004,Unpredictive
6,house_ownership,0.008146,Unpredictive
2,experience_bin,0.013336,Unpredictive
8,profession,0.025037,Weak
10,state,0.027764,Weak
